In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from dateutil.relativedelta import relativedelta

In [ ]:
expt_name = 'LS_OLv8_M36'
expt_name_da = 'LS_DAv8_M36'

start_date = datetime(2020, 1, 2)
end_date = datetime(2020, 1, 6)

start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

In [ ]:
def read_obsfcstana_extend_datetime(path, file_name, printflag=False):
    # Define precisions
    int_precision = 'int32'
    float_precision = 'float32'
    logical_precision = 'int32'

    # Initialize lists for outputs
    date_time_list = []
    obs_assim_list = []
    obs_species_list = []
    obs_tilenum_list = []
    obs_lon_list = []
    obs_lat_list = []
    obs_obs_list = []
    obs_obsvar_list = []
    obs_fcst_list = []
    obs_fcstvar_list = []
    obs_ana_list = []
    obs_anavar_list = []

    machfmt = 'b'
    file_ext = '.bin'
    # Build full file paths (note: file already includes the path)
    files = [os.path.join(root, file) 
             for root, dirs, files in os.walk(path) 
             for file in files if file.startswith(file_name) and file.endswith(file_ext)]

    if printflag:
        print(files)

    mode = 'rb' if machfmt == 'b' else 'rl'

    for file in files:
        with open(file, mode) as ifp:  # file already includes the path
            if printflag:
                print('Reading file', file, '...')
            
            # Read header and time stamp data
            _ = np.fromfile(ifp, int_precision, 1)  # fortran_tag
            N_obs = int(np.fromfile(ifp, int_precision, 1)[0])
            # Read time components
            year    = np.fromfile(ifp, int_precision, 1)
            month   = np.fromfile(ifp, int_precision, 1)
            day     = np.fromfile(ifp, int_precision, 1)
            hour    = np.fromfile(ifp, int_precision, 1)
            minute  = np.fromfile(ifp, int_precision, 1)
            second  = np.fromfile(ifp, int_precision, 1)
            dofyr   = np.fromfile(ifp, int_precision, 1)
            pentad  = np.fromfile(ifp, int_precision, 1)
            _ = np.fromfile(ifp, int_precision, 1)  # fortran_tag

            # Ensure all variables are scalars
            year = int(year.item()) if isinstance(year, (np.integer, np.ndarray)) else year
            month = int(month.item()) if isinstance(month, (np.integer, np.ndarray)) else month
            day = int(day.item()) if isinstance(day, (np.integer, np.ndarray)) else day
            hour = int(hour.item()) if isinstance(hour, (np.integer, np.ndarray)) else hour
            minute = int(minute.item()) if isinstance(minute, (np.integer, np.ndarray)) else minute
            second = int(second.item()) if isinstance(second, (np.integer, np.ndarray)) else second
            # Create a single datetime object for the timestamp info
            date_time_tmp = datetime(year, month, day, hour, minute, int(second))
            date_time_list.extend([date_time_tmp] * N_obs) 

            # Read observation assimilation flag
            _ = np.fromfile(ifp, int_precision, 1)
            tmp_data = np.fromfile(ifp, logical_precision, N_obs)
            _ = np.fromfile(ifp, int_precision, 1)
            # Vectorized conversion: nonzero becomes 1, else 0.
            tmp_data2 = (tmp_data != 0).astype(np.int32).reshape(-1, 1)
            obs_assim_list.append(tmp_data2)

            # Read species information
            _ = np.fromfile(ifp, int_precision, 1)
            obs_species_list.append(np.fromfile(ifp, int_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)
            
            # Read tile number information
            _ = np.fromfile(ifp, int_precision, 1)
            obs_tilenum_list.append(np.fromfile(ifp, int_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)

            # Read longitude
            _ = np.fromfile(ifp, int_precision, 1)
            obs_lon_list.append(np.fromfile(ifp, float_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)

            # Read latitude
            _ = np.fromfile(ifp, int_precision, 1)
            obs_lat_list.append(np.fromfile(ifp, float_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)
            
            # Read observation value
            _ = np.fromfile(ifp, int_precision, 1)
            obs_obs_list.append(np.fromfile(ifp, float_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)

            # Read observation variance
            _ = np.fromfile(ifp, int_precision, 1)
            obs_obsvar_list.append(np.fromfile(ifp, float_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)

            # Read forecast value
            _ = np.fromfile(ifp, int_precision, 1)
            obs_fcst_list.append(np.fromfile(ifp, float_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)

            # Read forecast variance
            _ = np.fromfile(ifp, int_precision, 1)
            obs_fcstvar_list.append(np.fromfile(ifp, float_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)

            # Read analysis value
            _ = np.fromfile(ifp, int_precision, 1)
            obs_ana_list.append(np.fromfile(ifp, float_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)

            # Read analysis variance
            _ = np.fromfile(ifp, int_precision, 1)
            obs_anavar_list.append(np.fromfile(ifp, float_precision, N_obs))
            _ = np.fromfile(ifp, int_precision, 1)

    # After processing all files, concatenate lists into numpy arrays
    obs_assim = np.concatenate(obs_assim_list) if obs_assim_list else np.array([])
    obs_species = np.concatenate(obs_species_list) if obs_species_list else np.array([])
    obs_tilenum = np.concatenate(obs_tilenum_list) if obs_tilenum_list else np.array([])
    obs_lon = np.concatenate(obs_lon_list) if obs_lon_list else np.array([])
    obs_lat = np.concatenate(obs_lat_list) if obs_lat_list else np.array([])
    obs_obs = np.concatenate(obs_obs_list) if obs_obs_list else np.array([])
    obs_obsvar = np.concatenate(obs_obsvar_list) if obs_obsvar_list else np.array([])
    obs_fcst = np.concatenate(obs_fcst_list) if obs_fcst_list else np.array([])
    obs_fcstvar = np.concatenate(obs_fcstvar_list) if obs_fcstvar_list else np.array([])
    obs_ana = np.concatenate(obs_ana_list) if obs_ana_list else np.array([])
    obs_anavar = np.concatenate(obs_anavar_list) if obs_anavar_list else np.array([])

    return (date_time_list, obs_assim, obs_species, obs_tilenum, obs_lon, obs_lat, 
            obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar)


In [ ]:
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from helper.read_GEOSldas import read_ObsFcstAna, read_tilecoord, read_obs_param

def compute_monthly_stats_OL(expdir, expid, domain, this_month, tc, obs_param, var_list, sub_expdir, sub_expid):

    n_tile = tc['N_tile']
    n_spec = len(obs_param)

    start_time = this_month.replace(day=1,hour=3) 
    end_time = start_time + relativedelta(months=1)

    data_sum = {}
    data2_sum = {}

    N_data = np.zeros((n_tile, n_spec))
    oxf_sum = np.zeros((n_tile, n_spec))
    oxa_sum = np.zeros((n_tile, n_spec))
    fxa_sum = np.zeros((n_tile, n_spec))

    for var in var_list:
        data_sum[var] = np.zeros((n_tile, n_spec))
        data2_sum[var] = np.zeros((n_tile, n_spec))

    date_time = start_time
    while date_time < end_time:

        fname = expdir + expid + '/output/' + domain + '/ana/ens_avg/Y' + \
                date_time.strftime('%Y') + '/M' + \
                date_time.strftime('%m') + '/' + \
                expid + '.ens_avg.ldas_ObsFcstAna.' + \
                date_time.strftime('%Y%m%d_%H%M') + 'z.bin'

        sub_fname = sub_expdir + sub_expid + '/output/' + domain + '/ana/ens_avg/Y' + \
                    date_time.strftime('%Y') + '/M' + \
                    date_time.strftime('%m') + '/' + \
                    sub_expid + '.ens_avg.ldas_ObsFcstAna.' + \
                    date_time.strftime('%Y%m%d_%H%M') + 'z.bin'

        print("fname: ", fname)

        OFA = read_ObsFcstAna(fname)

        # Check uniqueness of (tilenum, species) in OFA
        combined_keys = OFA['obs_tilenum'] * 100 + OFA['obs_species'].astype(int)
        n_total = len(combined_keys)
        n_unique = len(np.unique(combined_keys))
        if n_total != n_unique:
            print(f"[WARNING] {n_total - n_unique} duplicate (tilenum, species) combinations found in {fname}")
        OFA_sub = read_ObsFcstAna(sub_fname)

        # Check uniqueness of (tilenum, species) in OFA_sub
        combined_keys_sub = OFA_sub['obs_tilenum'] * 100 + OFA_sub['obs_species'].astype(int)
        n_total_sub = len(combined_keys_sub)
        n_unique_sub = len(np.unique(combined_keys_sub))
        if n_total_sub != n_unique_sub:
            print(f"[WARNING] {n_total_sub - n_unique_sub} duplicate (tilenum, species) combinations found in {sub_fname}")

        if len(OFA['obs_tilenum']) > 0 and len(OFA_sub['obs_tilenum']) > 0:

            # Build unique identifiers for matching using tilenum and species
            tilenum = OFA['obs_tilenum']
            species = OFA['obs_species'].astype(int)
            unique_id = tilenum * 100 + species

            tilenum_sub = OFA_sub['obs_tilenum']
            species_sub = OFA_sub['obs_species'].astype(int)
            unique_id_sub = tilenum_sub * 100 + species_sub

            # Find matching indices
            common_ids, idx_main, idx_sub = np.intersect1d(unique_id, unique_id_sub, return_indices=True)
            if len(common_ids) == 0:
                date_time += timedelta(seconds=10800)
                continue

            # Subset main data
            for var in var_list:
                OFA[var] = OFA[var][idx_main]
            OFA['obs_species'] = species[idx_main]
            OFA['obs_lat'] = OFA['obs_lat'][idx_main]
            OFA['obs_lon'] = OFA['obs_lon'][idx_main]
            OFA['obs_tilenum'] = tilenum[idx_main]
            if 'obs_assim' in OFA:
                OFA['obs_assim'] = OFA['obs_assim'][idx_main]

            # Replace obs_obs with subset values
            OFA['obs_obs'] = OFA_sub['obs_obs'][idx_sub]

            # Initialize full size variable to keep values
            data_tile = {var: np.full((n_tile, n_spec), np.nan) for var in var_list}

            for ispec in np.arange(n_spec):
                this_species = int(obs_param[ispec]['species'])
                masked_data = {}
                if obs_param[ispec]['assim'] == 'T':
                    mask = np.logical_and(OFA['obs_species'] == this_species, OFA['obs_assim'] == 1)
                else:
                    mask = OFA['obs_species'] == this_species

                lat = OFA['obs_lat'][mask]
                lon = OFA['obs_lon'][mask]
                tilenum = OFA['obs_tilenum'][mask]

                for var in var_list:
                    masked_data[var] = OFA[var][mask]

                tile_idx = np.where(np.isin(tc['tile_id'], tilenum))[0]

                for var in var_list:
                    data_tile[var][tile_idx, ispec] = masked_data[var]

            is_valid = ~np.isnan(data_tile['obs_obs'])
            N_data[is_valid] += 1
            oxf_sum[is_valid] += data_tile['obs_obs'][is_valid] * data_tile['obs_fcst'][is_valid]
            oxa_sum[is_valid] += data_tile['obs_obs'][is_valid] * data_tile['obs_ana'][is_valid]
            fxa_sum[is_valid] += data_tile['obs_fcst'][is_valid] * data_tile['obs_ana'][is_valid]
            for var in var_list:
                data_sum[var][is_valid] += data_tile[var][is_valid]
                data2_sum[var][is_valid] += data_tile[var][is_valid] ** 2

        date_time = date_time + timedelta(seconds=10800)

    return N_data, data_sum, data2_sum, oxf_sum, oxa_sum, fxa_sum

if __name__ == '__main__':
    date_time = datetime(2015,5,1)
    expdir = '/gpfsm/dnb05/projects/p51/SMAP_Nature/'
    expid = 'SPL4SM_Vv8010_OPENLOOP'
    sub_expdir = '/gpfsm/dnb05/projects/p51/SMAP_Nature/'
    sub_expid = 'SPL4SM_Vv8010_DA_RUN'
    domain = 'SMAP_EASEv2_M09_GLOBAL'
    var_list = ['obs_obs', 'obs_obsvar', 'obs_fcst', 'obs_fcstvar', 'obs_ana', 'obs_anavar']
    ftc = expdir+expid+'/output/'+domain+'/rc_out/'+expid+'.ldas_tilecoord.bin'
    tc = read_tilecoord(ftc)

    fop = expdir+expid+'/output/'+domain+'/rc_out/Y2015/M04/'+expid+'.ldas_obsparam.20150401_0000z.txt'
    obs_param = read_obs_param(fop)

    N_data, data_sum, data2_sum, oxf_sum, oxa_sum, fxa_sum = \
           compute_monthly_stats_OL(expdir, expid, domain, date_time, tc, obs_param, var_list, sub_expdir, sub_expid)
